In [1]:
#Mounting Google Drive from Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Changing the current working directory to the Google Drive
%cd /content/drive/My Drive/MLDL2024_project1-Enrico

/content/drive/My Drive/MLDL2024_project1-Enrico


In [3]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 929.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=035b97b4f83ac1966728c6589eb7b46c6d0f3a24c68c639aafe64cf677ee1b20
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=0f08bf896bc203e6fb8c1dddf21b07e3a8dd6af2cf00cd5a0e37b801e51b0ce2
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [4]:
import torch
from models.deeplabv2.deeplabv2 import get_deeplab_v2
from models.bisenet.build_bisenet import BiSeNet
from utils import test_latency_FPS, test_FLOPs_params

In [5]:
#Set device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

#Set the manual seeds
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

#Set training parameters
cityscapes_height, cityscapes_width = (64, 128)

Using device: cuda


In [6]:
DeepLabV2_model = get_deeplab_v2().to(device)

#Test latency and FPS
print(test_latency_FPS(DeepLabV2_model, device, cityscapes_height, cityscapes_width))

#Test FLOPs and number of parameters
print(test_FLOPs_params(DeepLabV2_model, device, cityscapes_height, cityscapes_width))

Deeplab pretraining loading...
Mean latency: 0.0215 +/- 0.0353 seconds 
Mean FPS: 49.18 +/- 2.97 frames per second
| module                         | #parameters or shape   | #flops     |
|:-------------------------------|:-----------------------|:-----------|
| model                          | 43.901M                | 54.799G    |
|  conv1                         |  9.408K                |  0.154G    |
|   conv1.weight                 |   (64, 3, 7, 7)        |            |
|  bn1                           |  0.128K                |  5.243M    |
|   bn1.weight                   |   (64,)                |            |
|   bn1.bias                     |   (64,)                |            |
|  layer1                        |  0.216M                |  0.988G    |
|   layer1.0                     |   75.008K              |   0.345G   |
|    layer1.0.conv1              |    4.096K              |    18.383M |
|    layer1.0.bn1                |    0.128K              |    1.436M  |
|    laye

In [7]:
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

#Test latency and FPS
print(test_latency_FPS(BiSeNet_model, device, cityscapes_height, cityscapes_width))

#Test FLOPs and number of parameters
print(test_FLOPs_params(BiSeNet_model, device, cityscapes_height, cityscapes_width))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 176MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 210MB/s]


Mean latency: 0.0064 +/- 0.0089 seconds 
Mean FPS: 163.88 +/- 9.46 frames per second
| module                                      | #parameters or shape   | #flops     |
|:--------------------------------------------|:-----------------------|:-----------|
| model                                       | 12.582M                | 3.265G     |
|  saptial_path                               |  0.371M                |  0.641G    |
|   saptial_path.convblock1                   |   1.856K               |   33.554M  |
|    saptial_path.convblock1.conv1            |    1.728K              |    28.312M |
|    saptial_path.convblock1.bn               |    0.128K              |    5.243M  |
|   saptial_path.convblock2                   |   73.984K              |   0.305G   |
|    saptial_path.convblock2.conv1            |    73.728K             |    0.302G  |
|    saptial_path.convblock2.bn               |    0.256K              |    2.621M  |
|   saptial_path.convblock3                   |   0.295